In [42]:
import pandas as pd
from numpy import array
import numpy as np
import math
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [43]:
train =pd.read_csv('Train.csv')
pressure_list = train['Pressure (KPa)'].tolist()
temp_list = train['Air temperature (C)'].tolist()
pressure_list_without_nan = []
for i in range(len(pressure_list)):
    if (np.isnan(pressure_list[i])):
        pressure_list_without_nan.append(pressure_list_without_nan[i - 24*12])
    else:
        pressure_list_without_nan.append(pressure_list[i])
temp_list_without_nan = []
for i in range(len(temp_list)):
    if (np.isnan(temp_list[i])):
        temp_list_without_nan.append(temp_list_without_nan[i - 24*12])
    else:
        temp_list_without_nan.append(temp_list[i])


In [44]:
def firstNanIndex(listfloats, k):
    for i, item in enumerate(listfloats):
        if i > k and math.isnan(item) == True:
            return i
        
def firstNonNan(listfloats, j):
    bol = False
    for i, item in enumerate(listfloats[j:]):
        if math.isnan(item) == False:
            bol = True
            return i+j, item
    if (bol == False):
        return len(listfloats) -1 , listfloats[len(listfloats)-1]
    
def cleanSerie(columnName):
    field = train[columnName].tolist()
    cleanedList = [x for x in field if (math.isnan(x) == False)]
    return cleanedList

                


In [45]:
field1_without_Nan = cleanSerie('Soil humidity 1')
field1_without_Nan = field1_without_Nan[:-3]
field1 = train['Soil humidity 1'].tolist()
index1 = firstNanIndex(field1, 0)
field1 = field1[:index1+ 1153]

field2_without_Nan = cleanSerie('Soil humidity 2')
field2_without_Nan = field2_without_Nan[:-3]
field2 = train['Soil humidity 2'].tolist()
index2 = firstNanIndex(field2, 0)
field2 = field2[:index2+ 1747]

field3_without_Nan = cleanSerie('Soil humidity 3')
field3_without_Nan = field3_without_Nan[:-3]
field3 = train['Soil humidity 3'].tolist()
index3 = firstNanIndex(field3, 0)
field3 = field3[:index3+ 1153]

field4_without_Nan = cleanSerie('Soil humidity 4')
field4_without_Nan = field4_without_Nan[:-5]
field4 = train['Soil humidity 4'].tolist()
index4 = firstNanIndex(field4, 0)
field4 = field4[:index4+ 1729]


In [52]:
def prepare_data(train,field_number, field_without_Nan, preds_horizon):
    target = train[['Pressure (KPa)','Air temperature (C)', 'Soil humidity ' + str(field_number),'Irrigation field ' +str(field_number)]][:len(field_without_Nan)]
    target['last'] = target[['Soil humidity ' + str(field_number)]].shift(1)
   
    target['diff'] = target['Soil humidity ' + str(field_number)] - target['last']
   
    target['pressure'] = target['Pressure (KPa)']   
    target['pressure_last'] = target[['pressure']].shift(1)
    target['diff_pressure'] = target['pressure'] - target['pressure_last']    
   
    target['temperature'] = target['Air temperature (C)']
    target['temperature_last'] = target[['temperature']].shift(1)
    target['diff_temperature'] = target['temperature'] - target['temperature_last']
    
   
    
    
    del target['Air temperature (C)']
    del target['temperature_last']
    del target['pressure_last']   
    del target['temperature']
    del target['pressure']   
    del target['Pressure (KPa)']
   

  
    target['irrigation_now'] = train['Irrigation field ' +str(field_number)][:len(field_without_Nan)]
    X = target.dropna()
   
    
    del X['Soil humidity ' + str(field_number)]
    del X['last']
    
   
  
    Y = X['diff']
    
    del X['Irrigation field ' +str(field_number)]   
    del X['diff']

    return X, Y


In [48]:
def train_xgboost(data_X, data_Y, random_state=0):
    my_imputer = SimpleImputer()
    final_train = my_imputer.fit_transform(data_X)
    train_X, test_X, train_y, test_y = train_test_split(final_train, data_Y, test_size=0.1, random_state=random_state)
    my_model = XGBRegressor(n_estimators=1500, learning_rate=0.1, random_state=random_state)
    my_model.fit(train_X, train_y, early_stopping_rounds=5,eval_metric=["rmse"], eval_set=[(test_X, test_y)], verbose=False)
    return my_model


In [49]:
def findStep(last_prediction, moisture_list, index):
    j = index
    while(np.isnan(moisture_list[j])):
        j =j+1
    step = (moisture_list[j] - last_prediction ) /(j-index)
    return step
    


In [50]:
def get_Predictions(model, field_number, field_without_Nan, field, train_last_index, preds_horizon):
    moisture = field[len(field_without_Nan)-1:]
    irrigationFrame = train[['Irrigation field ' +str(field_number)]][len(field_without_Nan)-1:len(field_without_Nan)+preds_horizon]
    

    preds = []
    k=0
    index=-1
    bol = False
    for i in range(preds_horizon):
        index= index + 1

        temperature = temp_list_without_nan[train_last_index + index]
        temperature_last = temp_list_without_nan[train_last_index + index - 1]
        temperature_diff = temperature - temperature_last
        
        pressure = pressure_list_without_nan[train_last_index + index]
        pressure_last = pressure_list_without_nan[train_last_index + index - 1]
        pressure_diff = pressure - pressure_last
        
        irrigation = irrigationFrame.iloc[i+1]['Irrigation field ' + str(field_number)]
        if(np.isnan(irrigation)):
            irrigation =0

        if(irrigation == 0):
            bol = False
            x= np.array([pressure_diff, temperature_diff, irrigation])
            x = x[np.newaxis,...]
            prediction = model.predict(x) 
        if(irrigation == 1 and index ==0):
            x= np.array([pressure_diff, temperature_diff, irrigation])
            x = x[np.newaxis,...]
            prediction = model.predict(x) 
        if(irrigation == 1 and index > 0):
            if(bol == False):
                step =findStep(preds[-1], moisture, index)
                prediction = [step]
                bol = True
            else:
                prediction = [step]                
     
        if np.isnan(moisture[i]):
            preds.append(prediction[0]+ preds[-1])
            k=k+1
        else:
            preds.append(prediction[0]+ moisture[i])
            k=0
            
        k=k+1
    return preds

In [53]:
seed = 90973
data_X, data_Y = prepare_data(train,1, field1_without_Nan, 1153)
model1 = train_xgboost(data_X, data_Y, seed)
preds1 = get_Predictions(model1,1, field1_without_Nan,field1, 8914, 1153)

data_X, data_Y = prepare_data(train,2, field2_without_Nan, 1747)
model2 = train_xgboost(data_X, data_Y, seed)
preds2 = get_Predictions(model2,2, field2_without_Nan,field2, 26301, 1747)

data_X, data_Y = prepare_data(train,3, field3_without_Nan, 1153)
model3 = train_xgboost(data_X, data_Y, seed)
preds3 = get_Predictions(model3,3, field3_without_Nan,field3, 16083, 1153)

data_X, data_Y = prepare_data(train,4, field4_without_Nan, 1729)
model4 = train_xgboost(data_X, data_Y, seed)
preds4 = get_Predictions(model4,4, field4_without_Nan,field4, 26301, 1729)



/home/havensen/.local/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[19:18:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/havensen/.local/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[19:18:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/havensen/.local/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[19:18:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/havensen/.local/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[19:18:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [54]:
total =[]
total.extend(preds1)
total.extend(preds2)
total.extend(preds3)
total.extend(preds4)

In [55]:
submission =pd.read_csv('SampleSubmission.csv')
submission['Values']= total
submission.to_csv('submission.csv', index=False)
